In [1]:
!pip install -qU langchain-community duckduckgo-search langchain-google-genai


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.3/53.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.6/426.6 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.1/489.1 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 234.9/234.9 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 1.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires google-aut

In [9]:
!pip install -U ddgs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 5.3 MB/s eta 0:00:00


In [12]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.tools import DuckDuckGoSearchResults, DuckDuckGoSearchRun
from langchain_core.runnables import RunnableBranch, RunnableLambda, RunnableParallel
from google.colab import userdata
from langchain.agents import create_agent
from datetime import datetime
from langchain_core.tools import tool
import requests

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite",google_api_key=userdata.get('Assignments'))

print("Libraries and API KEY Integrated Succesfully.")

Libraries and API KEY Integrated Succesfully.


**Intent Extraction**

In [13]:
intent_prompt = PromptTemplate(input_variables=["query"]
    template="""
You are a PRO news assistant.

From the user query below, extract:
1. topic : short phrase
2. Information Type : one of [headlines, deep_dive, mixed]

Return strictly in this format:
topic: <topic>
type: <headlines | deep_dive | mixed>

User Query:
{query}
""",

)



**DuckDuckGo Search Tool**

In [11]:
search_tool = DuckDuckGoSearchResults(output_format="list")


In [23]:
def intent_ext(text):
    lines = text.splitlines()
    topic = lines[0].split("topic:")[1].strip()
    info_type = lines[1].split("type:")[1].strip()
    return {"topic": topic, "type": info_type}
def context_ext(results, max_items=5):
    context = ""
    for i, item in enumerate(results[:max_items], start=1):
        context += (
            f"Headline {i}: {item['title']}\n"
            f"Description: {item['snippet']}\n"
            f"Link: {item['link']}\n\n"
        )
    return context


In [19]:
# Headline Prompt
headlines_prompt = PromptTemplate(input_variables=["context"],
    template="""
You are a PRO news assistant.

Given the following news articles, generate 3–5 concise bullet-point headlines.
Each bullet:
1)One sentence
2)Brief explanation
3)Optionally include the most relevant link in brackets

Articles:
{context}
""",

)
# Deep_Dive Prompt
deep_dive_prompt = PromptTemplate(input_variables=["context"],
    template="""
You are a PRO news research assistant.

Using the articles below, write a clear, beginner-friendly summary (2–4 short paragraphs)
explaining:
1)What is happening
2)Why it matters
3)Key dates, numbers, or names

No fluff. Be factual and clear.

Articles:
{context}
"""

)
# Mixed Prompt
mixed_prompt = PromptTemplate(input_variables=["context"],
    template="""
You are a PRO news assistant.

First, generate 3 short bullet-point headlines.
Then, write a brief 1–2 paragraph summary combining the key ideas.

Articles:
{context}
"""
)



In [31]:
intent_chain = intent_prompt | llm | StrOutputParser()
headlines_chain = headlines_prompt | llm | StrOutputParser()
deep_dive_chain = deep_dive_prompt | llm | StrOutputParser()
mixed_chain = mixed_prompt | llm | StrOutputParser()
runnable_branch = RunnableBranch(
    (lambda x: x["type"] == "headlines", headlines_chain),
    (lambda x: x["type"] == "deep_dive", deep_dive_chain),
    (lambda x: x["type"] == "mixed", mixed_chain),
    mixed_chain
)


In [33]:
user_query = input("Enter your query: ")
def news_assistant(query):
   intent_text = intent_chain.invoke({"query": query})
   intent = intent_ext(intent_text)
   results = search_tool.invoke(intent["topic"])
   context = context_ext(results)
   response = runnable_branch.invoke({
       "context": context,
       "type": intent["type"]
   })
   return response
print(news_assistant(user_query))
print(news_assistant("Current inflation news in India"))
print(news_assistant("Recent breakthroughs in cancer research"))


Enter your query: latest news in AI Industry
Here are 3 concise bullet-point headlines based on the provided articles:

*   **AI Market Poised for Explosive Growth:** Projections indicate the global AI market could reach over $1.7 trillion by 2032, with significant annual growth rates and a booming chip sector. [https://www.fortunebusinessinsights.com/industry-reports/artificial-intelligence-market-100114]
*   **AI's Economic Impact and Future Questions:** The rapid expansion of AI is driving tech industry growth and economic development, but concerns about a potential bubble and the responsible guidance of this transformation are surfacing. [https://www.pbs.org/newshour/show/whats-next-for-ai-and-has-its-explosive-growth-in-2025-created-a-bubble]
*   **AI Set to Reshape Industries, Focus on Human Well-being:** Experts agree AI will fundamentally transform various sectors, with the critical challenge being how to steer this evolution to ensure human security and well-being. [https://ww